In [109]:
pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [110]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from llama_index.core import PromptTemplate
from huggingface_hub import InferenceClient


In [111]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [112]:
import nest_asyncio

nest_asyncio.apply()

In [113]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set local LLM for embeddings

In [114]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Ollama)

In [115]:
llm = Ollama(model="tinyllama:latest", request_timeout=120.0)
Settings.llm = llm
Settings.chunk_size = 512

## Test

In [155]:
messages = [
    ChatMessage(
        role="system", content="Tu es un consultant spécialisé dans la gestion des risques"
    ),
    ChatMessage(role="user", content=""),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
assistant: Je ne comprends pas votre question. J'aiimerais vous parler de programmation en anglais si cela vous aide. Je suis un professeur d'ingénierie informatique, mais je ne fais que des cours à temps partiel pour les étudiants étrangers. Si vous voulez avoir une conversation sur ce sujet avec moi, il faut que nous soinions de définir ce qui est votre question. Vous pouvez par exemple essayer d'expliquer quelque chose que vous avez déjà lu, ou me dire ce que souhaitez savoir sur un sujet en général. Si vous voulez être aidé dans la recherche de ma documentation pour expliquer ou apprendre plus sur une telle question, je suis là pour

# Test LLM

# Load storage contexts

## Load vector storage context

In [156]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

# Load index

## Load vector index

In [188]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


# Set retriever

In [228]:
vector_chat_engine.reset()

In [229]:
custom_query_template = PromptTemplate(
    "Nous avons fourni des informations contextuelles ci-dessous. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "En utilisant ces informations, veuillez répondre à la question suivante : {query_str}\n"
)


In [234]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
       """ 
Je souhaite te former afin que tu deviennes un expert dans la réalisation de cartographies des risques. La cartographie des risques, également connue sous le nom de cartographie de l'aléa, est un outil stratégique permettant de visualiser, analyser et évaluer les risques en fonction de leurs caractéristiques spatiales. Elle s'applique à différentes échelles et permet de représenter :



1. **La répartition spatiale des aléas** : Identifier les zones où les risques sont susceptibles de se produire (par exemple, zones de crue, sismicité, risques industriels).

2. **La répartition spatiale des enjeux** : Localiser les biens, populations ou infrastructures pouvant être affectés par ces risques (exemples : habitations, infrastructures critiques, zones agricoles).

3. **Les vulnérabilités spatiales** : Identifier les zones particulièrement sensibles ou exposées aux risques, en fonction de critères géographiques, sociaux et économiques (par exemple, zones d'habitat dense ou zones peu équipées en infrastructure).

4. **Combinaison des trois facteurs** : Une cartographie intégrée qui combine les aléas, les enjeux et les vulnérabilités afin d'identifier les zones à risque maximal.



### Exemples d’utilisations :

- **Répartition des niveaux de dangerosité** : Cartographier les différents niveaux de danger (par exemple, zones à faible, moyen ou fort risque) en fonction des aléas étudiés.

- **Mesures de prévention et gestion des risques** : Identifier les zones nécessitant des actions de prévention, telles que la construction de digues, le renforcement des infrastructures, ou la planification d'évacuation.

- **Restrictions d'usage des terrains** : Utiliser la cartographie pour appliquer des servitudes d'utilité publique, telles que les Plans de Prévention des Risques (PPR), qui régulent ou limitent l'utilisation des terrains dans les zones à risque.



L'objectif de cette formation est de te fournir une maîtrise complète des concepts fondamentaux de la cartographie des risques, ainsi que les outils nécessaires pour mener une analyse approfondie et pertinente, facilitant ainsi la prise de décision pour la gestion des risques.


"""


    ),
    text_qa_template=custom_query_template,
    verbose=False,
    similarity_top_k=4
    
)

In [235]:
response_stream = vector_chat_engine.stream_chat("""Effectue une cartographie des risques en utilisant les éléments disponibles dans le contexte actuel. Identifie et décris les risques potentiels en tenant compte des facteurs internes (ressources humaines, technologies, processus internes) ainsi que des facteurs externes (environnementaux, économiques, réglementaires). Pour chaque risque, évalue la probabilité de son occurrence et son impact potentiel sur l’organisation. Classe les risques selon leur priorité, en mettant l'accent sur les mesures de gestion proactive nécessaires.""")

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Effectue une cartographie des risques en utilisant les éléments disponibles dans le contexte actuel. Identifie et décris les risques potentiels en tenant compte des facteurs internes (ressources humaines, technologies, processus internes) ainsi que des facteurs externes (environnementaux, économiques, réglementaires). Pour chaque risque, évalue la probabilité de son occurrence et son impact potentiel sur l’organisation. Classe les risques selon leur priorité, en mettant l'accent sur les mesures de gestion proactive nécessaires.
Condensed question: Effectue une cartographie des risques en utilisant les éléments disponibles dans le contexte actuel. Identifie et décris les risques potentiels en tenant compte des facteurs internes (ressources humaines, technologies, processus internes) ainsi que des facteurs externes (environnementaux, économiques, réglementaires). Pour chaque risque, évalue la probabilité de son o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [236]:
generate = response_stream.print_response_stream()

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Pour une cartographie des risques, il est nécessaire d'avoir une connaissance approfondie du contexte actuel. L'analyse doit être prise en compte des facteurs internes (ressources humaines, technologies, processus internes) et des facteurs externes (environnementaux, économiques, réglementaires). Les risques potentiels doivent être déterminés à partir de l'ensemble des données disponibles sur la situation actuelle. Il est aussi utile d'éviter les risques récurrents, c'est-à-dire ceux qui ne présentent pas de risque potentiel d'atteindre leur point culminant.

Pour chaque risque, il convient de classer l'incident selon son impact sur l’o

In [237]:
# Afficher les chunks utilisés
for node in response_stream.source_nodes:
    print(node.node.text)
    

Les facteurs de risques du Groupe présentés ci‑dessous sont ceux identiﬁés comme étant dans le périmètre de l'ERM à la date 
du présent Document d'enregistrement universel, étant précisé que certaines informations présentées ci‑dessous et identiﬁées 
comme telles se rapportent spéciﬁquement à l'exercice 2023.
2.1. Méthodologie, description 
des/uni00A0principaux facteurs du risque 
et/uni00A0de/uni00A0leur/uni00A0maîtrise
Le Groupe exerce ses activités dans un environnement en transformation permanente. Il s’expose ainsi à des facteurs de 
risques, qui pourraient se traduire par des évènements dont la probabilité d’occurrence et/ou de gravité pourraient nuire à 
la réalisation de ses objectifs à court, moyen ou long terme./uni00A0
Méthodologie
La sensibilisation à la culture du risque est l'élément clef de 
la méthodologie, poiur laquelle une formation est 
dispensée à tous les acteurs du dispositif de gestion des 
risques. Le Groupe réalise annuellement une cartographie 
de tolérance 

In [207]:
query = "Réalise une cartographie des risques avec les éléments dont tu disposes"
query_engine = simple_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=8,
)

response = query_engine.query(query)

# Afficher les chunks utilisés
for node in response.source_nodes:
    print(node.node.text)
    print("---")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Indicateurs clefs de performance de nature ﬁnancière Article/uni00A0L./uni00A0225‑100‑1,/uni00A0I., 2° 
C.com
Chapitre introductif/uni00A0; 1.1.1 
à/uni00A01.1.6/uni00A0; 1.2/uni00A0; 1.5
1.3. Indicateurs clefs de performance de nature non ﬁnancière 
ayant trait à l’activité spéciﬁque de la Société et du Groupe, 
notamment les informations relatives aux questions 
d’environnement et de personnel
Article/uni00A0L./uni00A0225‑100‑1,/uni00A0I., 
2°C.com
4.1.3.1; 4.1.3.2; 4.1.1.1; 4.2; 4.3 
et 4.4
1.4. Événements importants survenus entre la date /uni00A0de clôture 
de/uni00A0l’exercice et la date à laquelle le rapport de gestion est 
établ

In [208]:
print(response)

du p ch ch du,  d: ch  and for plus: d de  d  dans [[ av ch ou es p ch&# s du par d&j/s no dans des nots & und b byd

p ch  at the  du du dues de s undd ch des: for du not ju re not un ps {{i [[d and context av ch p and o ch du du clas d {{&#d js un d<thees the s un {{
pros chuns du deses and chles ch ete
desch
d
chp p.je
aup ls
fors
j
du g de la chju [[ch s pres context p:s des r p{{1 p im not l s un p l a not d s context es de d so un av a {{a f
vis


to
s ads un

j
au c 
ju
se ai de at of ch
the int avs
unc
s

prov
vi the
m
d w
